<a href="https://colab.research.google.com/github/aruna20200/Deeplearning-and-NLP/blob/master/Copy_of_Text_Classification_Machine_Learning_and_Basic_DNN_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis - Machine Learning and Basic Deep Neural Network Models

We have already discussed that sentiment analysis, also popularly known as opinion analysis or opinion mining is one of the most important applications of NLP. The key idea is to predict the potential sentiment of a body of text based on the textual content. In this sub-unit, we will be exploring supervised learning models. 

![](https://github.com/dipanjanS/nlp_workshop_dhs18/blob/master/Unit%2012%20-%20Project%209%20-%20Sentiment%20Analysis%20-%20Supervised%20Learning/sentiment_cover.png?raw=1)

Another way to build a model to understand the text content and predict the sentiment of the text based reviews is to use supervised machine learning. To be more specific, we will be using classification models for solving this problem. We will be building an automated sentiment text classification system in subsequent sections. The major steps to achieve this are mentioned as follows.

1.	Prepare train and test datasets (optionally a validation dataset)
2.	Pre-process and normalize text documents
3.	Feature Engineering 
4.	Model training
5.	Model prediction and evaluation

These are the major steps for building our system. Optionally the last step would be to deploy the model in your server or on the cloud. The following figure shows a detailed workflow for building a standard text classification system with supervised learning (classification) models.

![](https://github.com/dipanjanS/nlp_workshop_dhs18/blob/master/Unit%2012%20-%20Project%209%20-%20Sentiment%20Analysis%20-%20Supervised%20Learning/sentiment_classifier_workflow.png?raw=1)


In our scenario, documents indicate the movie reviews and classes indicate the review sentiments which can either be positive or negative making it a binary classification problem. We will build models using both traditional machine learning methods and newer deep learning in the subsequent sections. 

In [ ]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

     |████████████████████████████████| 266kB 35.7MB/s 
     |████████████████████████████████| 327kB 48.4MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85398 sha256=64fa6802cbc3defe41474b94a6443e4292bed15cc4492e2c09d616560f5a2456
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Load and View Dataset

In [ ]:
from google.colab import files 
  
  
uploaded = files.upload()

Saving Data_Train.xlsx to Data_Train.xlsx


In [ ]:
from google.colab import files 
  
  
uploaded = files.upload()

Saving Data_Test.xlsx to Data_Test.xlsx


In [ ]:
import pandas as pd

dataset = pd.read_excel('/content/Data_Train.xlsx')
test_dataset = pd.read_excel('/content/Data_Test.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7628 entries, 0 to 7627
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   STORY    7628 non-null   object
 1   SECTION  7628 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 119.3+ KB


In [ ]:
from google.colab import files 
  
  
uploaded = files.upload()

Saving Sample_submission.xlsx to Sample_submission.xlsx


In [ ]:
dataset.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


# Build Train and Test Datasets

In [ ]:
# build train and test datasets
sentiments= dataset['SECTION'].values
reviews = dataset['STORY'].values

train_reviews = dataset['STORY']
train_sentiments = dataset['SECTION']

test_reviews = test_dataset['STORY']

In [ ]:
to_categorical(train_sentiments)

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

# Text Wrangling & Normalization

In [ ]:
import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  
  return norm_docs

In [ ]:
%%time

norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

100%|██████████| 2748/2748 [00:00<00:00, 4205.01it/s]

CPU times: user 2.47 s, sys: 41.3 ms, total: 2.51 s
Wall time: 2.5 s


# Traditional Supervised Machine Learning Models

## Feature Engineering

In [ ]:
%%time

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# build BOW features on train reviews
cv = CountVectorizer(binary=False, min_df=5, max_df=1.0, ngram_range=(1,2))
cv_train_features = cv.fit_transform(norm_train_reviews)


# build TFIDF features on train reviews
tv = TfidfVectorizer(use_idf=True, min_df=5, max_df=1.0, ngram_range=(1,2),
                     sublinear_tf=True)
tv_train_features = tv.fit_transform(norm_train_reviews)

CPU times: user 4.85 s, sys: 20.4 ms, total: 4.87 s
Wall time: 4.87 s


In [ ]:
%%time

# transform test reviews into features
cv_test_features = cv.transform(norm_test_reviews)
tv_test_features = tv.transform(norm_test_reviews)

CPU times: user 864 ms, sys: 2.29 ms, total: 867 ms
Wall time: 870 ms


In [ ]:
print('BOW model:> Train features shape:', cv_train_features.shape, ' Test features shape:', cv_test_features.shape)
print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

BOW model:> Train features shape: (7628, 31941)  Test features shape: (2748, 31941)
TFIDF model:> Train features shape: (7628, 31941)  Test features shape: (2748, 31941)


## Model Training, Prediction and Performance Evaluation

### Try out Logistic Regression

The logistic regression model is actually a statistical model developed by statistician
David Cox in 1958. It is also known as the logit or logistic model since it uses the
logistic (popularly also known as sigmoid) mathematical function to estimate the
parameter values. These are the coefficients of all our features such that the overall loss
is minimized when predicting the outcome—

In [ ]:
%%time

# Logistic Regression model on BOW features
from sklearn.linear_model import LogisticRegression

# instantiate model
lr = LogisticRegression(penalty='l2', max_iter=500, C=1, solver='lbfgs', random_state=42)

# train model
lr.fit(cv_train_features, train_sentiments)

# predict on test data
lr_bow_predictions = lr.predict(cv_test_features)

CPU times: user 8.98 s, sys: 10.7 s, total: 19.7 s
Wall time: 10.1 s


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_bow_predictions), index=labels, columns=labels)

NameError: ignored

In [ ]:
%%time

# Logistic Regression model on TF-IDF features

# train model
lr.fit(tv_train_features, train_sentiments)

# predict on test data
lr_tfidf_predictions = lr.predict(tv_test_features)

In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_tfidf_predictions), index=labels, columns=labels)

### Try out Random Forest

Decision trees are a family of supervised machine learning algorithms that can represent
and interpret sets of rules automatically from the underlying data. They use metrics like
information gain and gini-index to build the tree. However, a major drawback of decision
trees is that since they are non-parametric, the more data there is, greater the depth of
the tree. We can end up with really huge and deep trees that are prone to overfitting. The
model might work really well on training data, but instead of learning, it just memorizes
all the training samples and builds very specific rules to them. Hence, it performs really
poorly on the test data. Random forests try to tackle this problem.

A random forest is a meta-estimator or an ensemble model that fits a number of
decision tree classifiers on various sub-samples of the dataset and uses averaging to
improve the predictive accuracy and control over-fitting. The sub-sample size is always
the same as the original input sample size, but the samples are drawn with replacement
(bootstrap samples). In random forests, all the trees are trained in parallel (bagging
model/bootstrap aggregation). Besides this, each tree in the ensemble is built from a
sample drawn with replacement (i.e., a bootstrap sample) from the training set. Also,
when splitting a node during the construction of the tree, the split that is chosen is no
longer the best split among all features. Instead, the split that is picked is the best split
among a random subset of the features. T

In [ ]:
%%time

# Random Forest model on BOW features
from sklearn.ensemble import RandomForestClassifier

# instantiate model
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)

# train model
rf.fit(cv_train_features, train_sentiments)

# predict on test data
rf_bow_predictions = rf.predict(cv_test_features)

In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, rf_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, rf_bow_predictions), index=labels, columns=labels)

In [ ]:
%%time

# Random Forest model on TF-IDF features

# train model
rf.fit(tv_train_features, train_sentiments)

# predict on test data
rf_tfidf_predictions = rf.predict(tv_test_features)

In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, rf_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, rf_tfidf_predictions), index=labels, columns=labels)

# Newer Supervised Deep Learning Models

In [ ]:
import gensim
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Dense
from sklearn.preprocessing import OneHotEncoder
from keras.utils.np_utils import to_categorical

## Prediction class label encoding

In [ ]:
# converting y data into categorical (one-hot encoding)
y_train = to_categorical(train_sentiments)
#y_test = to_categorical(y_test)

# tokenize train reviews & encode train labels
tokenized_train = [nltk.word_tokenize(text)
                       for text in norm_train_reviews]
y_train = le.fit_transform(y_train)
# tokenize test reviews & encode test labels
tokenized_test = [nltk.word_tokenize(text)
                      for text in norm_test_reviews]
#y_test = le.fit_transform(test_sentiments)

NameError: ignored

In [ ]:
# print class label encoding map and encoded labels
print('Sentiment class label map:', dict(zip(le.classes_, le.transform(le.classes_))))
print('Sample test label transformation:\n'+'-'*35,
      '\nActual Labels:', test_sentiments[:3], '\nEncoded Labels:', y_test[:3])

NameError: ignored

## Feature Engineering with word embeddings

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
%%time
# build word2vec model
w2v_num_features = 300
w2v_model = gensim.models.Word2Vec(tokenized_train, size=w2v_num_features, window=150,
                                   min_count=10, workers=4, iter=5)    

2021-04-13 13:10:40,848 : INFO : collecting all words and their counts
2021-04-13 13:10:40,850 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-04-13 13:10:41,007 : INFO : collected 40121 word types from a corpus of 821529 raw words and 7628 sentences
2021-04-13 13:10:41,008 : INFO : Loading a fresh vocabulary
2021-04-13 13:10:41,045 : INFO : effective_min_count=10 retains 6846 unique words (17% of original 40121, drops 33275)
2021-04-13 13:10:41,046 : INFO : effective_min_count=10 leaves 746491 word corpus (90% of original 821529, drops 75038)
2021-04-13 13:10:41,069 : INFO : deleting the raw counts dictionary of 40121 items
2021-04-13 13:10:41,073 : INFO : sample=0.001 downsamples 36 most-common words
2021-04-13 13:10:41,077 : INFO : downsampling leaves estimated 572095 word corpus (76.6% of prior 746491)
2021-04-13 13:10:41,094 : INFO : estimated required memory for 6846 words and 300 dimensions: 19853400 bytes
2021-04-13 13:10:41,095 : INFO : resettin

CPU times: user 43.6 s, sys: 133 ms, total: 43.7 s
Wall time: 23.2 s


In [ ]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [ ]:
# generate averaged word vector features from word2vec model
avg_wv_train_features = averaged_word2vec_vectorizer(corpus=tokenized_train, model=w2v_model,
                                                     num_features=w2v_num_features)
avg_wv_test_features = averaged_word2vec_vectorizer(corpus=tokenized_test, model=w2v_model,
                                                    num_features=w2v_num_features)

NameError: ignored

In [ ]:
print('Word2Vec model:> Train features shape:', avg_wv_train_features.shape, ' Test features shape:', avg_wv_test_features.shape)

## Modeling with deep neural networks 

### Building Deep neural network architecture

In [ ]:
from keras.layers import BatchNormalization

In [ ]:
def construct_deepnn_architecture(num_input_features):
    dnn_model = Sequential()
    dnn_model.add(Dense(512, input_shape=(num_input_features,), kernel_initializer='he_normal'))
    dnn_model.add(BatchNormalization())
    dnn_model.add(Activation('elu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(256, kernel_initializer='he_normal'))
    dnn_model.add(BatchNormalization())
    dnn_model.add(Activation('elu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(256, kernel_initializer='he_normal'))
    dnn_model.add(BatchNormalization())
    dnn_model.add(Activation('elu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(4))
    dnn_model.add(Activation('softmax'))

    dnn_model.compile(loss='categorical_crossentropy', optimizer='adam',                 
                      metrics=['accuracy'])
    return dnn_model

In [ ]:
w2v_dnn = construct_deepnn_architecture(num_input_features=w2v_num_features)

### Visualize sample deep architecture

In [ ]:
w2v_dnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               154112    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0

### Model Training, Prediction and Performance Evaluation

In [ ]:
batch_size = 100
w2v_dnn.fit(avg_wv_train_features, to_categorical(train_sentiments), epochs=50, batch_size=batch_size, 
            shuffle=True, validation_split=0.1, verbose=1)

Epoch 1/50
69/69 [==============================] - 4s 8ms/step - loss: 0.3629 - accuracy: 0.8763 - val_loss: 0.1999 - val_accuracy: 0.9345
Epoch 2/50
69/69 [==============================] - 0s 5ms/step - loss: 0.1938 - accuracy: 0.9374 - val_loss: 0.1575 - val_accuracy: 0.9410
Epoch 3/50
69/69 [==============================] - 0s 5ms/step - loss: 0.1745 - accuracy: 0.9435 - val_loss: 0.1384 - val_accuracy: 0.9554
Epoch 4/50
69/69 [==============================] - 0s 5ms/step - loss: 0.1644 - accuracy: 0.9459 - val_loss: 0.1419 - val_accuracy: 0.9567
Epoch 5/50
69/69 [==============================] - 0s 5ms/step - loss: 0.1647 - accuracy: 0.9445 - val_loss: 0.1316 - val_accuracy: 0.9620
Epoch 6/50
69/69 [==============================] - 0s 5ms/step - loss: 0.1601 - accuracy: 0.9467 - val_loss: 0.1568 - val_accuracy: 0.9515
Epoch 7/50
69/69 [==============================] - 0s 5ms/step - loss: 0.1716 - accuracy: 0.9434 - val_loss: 0.1426 - val_accuracy: 0.9541
Epoch 8/50
69/69 [==

In [ ]:
np.array(y_train )

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [ ]:
y_pred = w2v_dnn.predict_classes(avg_wv_test_features)
y_pred
#predictions =  np.argmax(y_pred)

NameError: ignored

In [ ]:
submission = pd.read_excel('/content/Sample_submission.xlsx')
submission['SECTION'] = y_pred
submission.to_excel('/content/Sample_submission_w2v_CNN.xlsx',index=False)

FileNotFoundError: ignored

In [ ]:
y_pred.max()

In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, predictions))
pd.DataFrame(confusion_matrix(test_sentiments, predictions), index=labels, columns=labels)

In [ ]:
import tensorflow as tf

t = tf.keras.preprocessing.text.Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t.fit_on_texts(norm_train_reviews)
t.word_index['<PAD>'] = 0

In [ ]:
VOCAB_SIZE = len(t.word_index)

In [ ]:
train_sequences = t.texts_to_sequences(norm_train_reviews)
test_sequences = t.texts_to_sequences(norm_test_reviews)
X_train = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=1000)
X_test = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=1000)

In [ ]:
EMBEDDING_DIM = 300 # dimension for dense embeddings for each token
LSTM_DIM = 128 # total LSTM units

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=1000))
model.add(tf.keras.layers.SpatialDropout1D(0.1))
model.add(tf.keras.layers.LSTM(LSTM_DIM, return_sequences=False))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam",
              metrics=["accuracy"])
model.summary()

In [ ]:
batch_size = 100
model.fit(X_train, to_categorical(train_sentiments), epochs=10, batch_size=batch_size, 
          shuffle=True, validation_split=0.1, verbose=1)

In [ ]:
train_sequences[1]

In [ ]:
X_train[1]


In [ ]:
predictions = model.predict_classes(X_test)
predictions[:10]

In [ ]:
submission = pd.read_excel('/content/Sample_submission.xlsx')
submission['SECTION'] = predictions
submission.to_excel('/content/Sample_submission_w2v_LSTM.xlsx',index=False)